In [2]:

from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf
from os import listdir
from PIL import Image
import glob, os
from os import listdir
from os.path import isfile, join
import numpy as np
from pandas import DataFrame
import pandas as pd
from matplotlib import pyplot as plt
#from __future__ import print_function

from SRC.transform import *
#estas son mis funciones traidas del jupyter contiguo

In [3]:
paths={"black":"INPUT/black/",
      "white":"INPUT/white/",
      "green":"INPUT/green/",
      "red":"INPUT/red/"}
images={}
for color,path in paths.items():
    images[color]=[]
    for im in listdir(path):
        if im.endswith("jpg"):
            #size=(64,64)
            a=resize_image(path+im,(64,64))
            images[color].append(np.asarray(a))
    images[color]=np.array(images[color])

In [5]:
X= np.concatenate(list(images.values()),axis=0)
y = np.concatenate([[k]* v.shape[0] for k,v in images.items()])

# Division entre la x y la y para test
### Ojo, normalmente convertiriamos el onehotencoder antes de esta division para ahorrarnos un paso

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
img_width, img_height = 64, 64
img_rows, img_cols = 64, 64

# Prepare data to feed the NN
num_classes = 4

# Ask keras which format to use depending on used backend and arrange data as expected
if K.image_data_format() == 'channels_first':
    X_train = x_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = x_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_width, img_height)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_width, img_height, 3)

# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
X_train = tf.keras.utils.normalize(X_train,axis=1)
X_test = tf.keras.utils.normalize(X_test,axis=1)
#X_train = X_train.astype('float32') / 255
#X_test = X_test.astype('float32') / 255

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (8388, 64, 64, 3)
8388 train samples
2097 test samples


In [8]:
#red neuronal propiamente dicha

In [9]:
img_width, img_height = 64, 64

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.Flatten())

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Proceso de cambio de las y para poder utilizarlas en el modelo. 
### Las necesitamos en el formato de array sin strings y getdummies no nos sirvio ya que 
### no podiamos obtener el color con posterioridad.

In [10]:
# importo aqui para que quede reflejado el modulo que utilizamos
from sklearn.preprocessing import OneHotEncoder

In [11]:
y_test


array(['black', 'red', 'black', ..., 'white', 'white', 'red'], dtype='<U5')

In [12]:
# Hacemos el reshape, luego onehot encoder, luego fit y por ultimo toarray
y_test= y_test.reshape(-1, 1)
y_test

array([['black'],
       ['red'],
       ['black'],
       ...,
       ['white'],
       ['white'],
       ['red']], dtype='<U5')

In [13]:
##3
enc = OneHotEncoder()
enc.fit(y_test)  

OneHotEncoder()

In [14]:
y_test = enc.transform(y_test).toarray()

In [15]:
#por fin y_test esta en la forma que queremos
y_test


array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [16]:
#ahora hago lo mismo con y Y_train

In [17]:
y_train

array(['red', 'black', 'black', ..., 'white', 'white', 'red'], dtype='<U5')

In [18]:
y_train= y_train.reshape(-1, 1)

In [19]:
enc = OneHotEncoder()
enc.fit(y_train)  

OneHotEncoder()

In [20]:
y_train = enc.transform(y_train).toarray()

In [21]:
y_train

array([[0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [22]:
#ha funcionado. Tenemos y_test e y_train en el formato que necesitabamos
# podiamos haberlo hecho antes de dividir el split del train pero he decidido hacerlo dos veces para practicar

In [23]:
enc.get_feature_names()

array(['x0_black', 'x0_green', 'x0_red', 'x0_white'], dtype=object)

In [24]:
enc.inverse_transform([[0,1,0,0]])

array([['green']], dtype='<U5')

In [25]:
# Codigo para para correr el modelo de red neuronal. 

In [26]:
batch_size = 100
epochs = 20

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))



Epoch 1/20
84/84 [==============================] - 12s 148ms/step - loss: 0.3625 - accuracy: 0.6964 - val_loss: 0.2200 - val_accuracy: 0.8340
Epoch 2/20
84/84 [==============================] - 14s 163ms/step - loss: 0.1955 - accuracy: 0.8652 - val_loss: 0.1601 - val_accuracy: 0.8908
Epoch 3/20
84/84 [==============================] - 15s 181ms/step - loss: 0.1403 - accuracy: 0.9053 - val_loss: 0.0966 - val_accuracy: 0.9356
Epoch 4/20
84/84 [==============================] - 15s 180ms/step - loss: 0.1038 - accuracy: 0.9314 - val_loss: 0.0774 - val_accuracy: 0.9442
Epoch 5/20
84/84 [==============================] - 15s 180ms/step - loss: 0.0827 - accuracy: 0.9498 - val_loss: 0.0715 - val_accuracy: 0.9475
Epoch 6/20
84/84 [==============================] - 14s 172ms/step - loss: 0.0741 - accuracy: 0.9535 - val_loss: 0.0628 - val_accuracy: 0.9590
Epoch 7/20
84/84 [==============================] - 15s 181ms/step - loss: 0.0679 - accuracy: 0.9565 - val_loss: 0.0529 - val_accuracy: 0.9652

In [ ]:
# PRUEBAS

In [31]:
X_test[45].shape
model.predict(np.array([X_test[2000]]))
#nos dice cual es la prediccion del modelo

array([[9.9999976e-01, 6.4836332e-09, 1.1157968e-09, 1.2618054e-12]],
      dtype=float32)

In [38]:
enc.get_feature_names()
# y hay que recordar que los colores marcados son:

array(['x0_black', 'x0_green', 'x0_red', 'x0_white'], dtype=object)

In [33]:
enc.inverse_transform([[9.9999976e-01, 6.4836332e-09, 1.1157968e-09, 1.2618054e-12]])
#nos da el resultado mas probable en una tuple. 


array([['black']], dtype='<U5')

In [37]:
X[2000].shape

(64, 64, 3)

In [40]:
model.predict(np.array([X_test[2000]]))

array([[9.9999976e-01, 6.4836332e-09, 1.1157968e-09, 1.2618054e-12]],
      dtype=float32)

In [39]:
enc.inverse_transform([[9.9999976e-01, 6.4836332e-09, 1.1157968e-09, 1.2618054e-12]])[0][0]
# le digo que me muestre solo el primer elemento, que es el color

'black'

In [41]:
X_test[2000]

array([[[0.15078995, 0.14719772, 0.14624406],
        [0.14945908, 0.14606556, 0.1452462 ],
        [0.15799422, 0.15385421, 0.15314105],
        ...,
        [0.11984021, 0.11786016, 0.11817176],
        [0.11361113, 0.11999259, 0.11899661],
        [0.11617704, 0.11853353, 0.11780818]],

       [[0.16084261, 0.1570109 , 0.15599366],
        [0.15705869, 0.15349262, 0.1526316 ],
        [0.15034934, 0.14640965, 0.145731  ],
        ...,
        [0.13114589, 0.12897905, 0.12932004],
        [0.12724447, 0.1333251 , 0.13246792],
        [0.120733  , 0.1230065 , 0.12233926]],

       [[0.14827678, 0.14965101, 0.14624406],
        [0.15199228, 0.14854125, 0.147708  ],
        [0.16309081, 0.15385421, 0.15561106],
        ...,
        [0.13792929, 0.13565038, 0.136009  ],
        [0.12951669, 0.12665885, 0.12797749],
        [0.14579079, 0.14313483, 0.14499468]],

       ...,

       [[0.09298713, 0.09567852, 0.09262124],
        [0.09372857, 0.09655181, 0.0935484 ],
        [0.09428687, 0

In [86]:
def car_colour (im):
    x = model.predict(np.array([im]))
    
    return x[0]

In [87]:
print (car_colour (X[600]))

[1. 0. 0. 0.]


In [88]:
enc.inverse_transform([[1. 0. 0. 0.]])[0][0]

SyntaxError: invalid syntax (<ipython-input-88-3e238f67da9e>, line 1)

In [118]:
# las imagenes deben ser metidas como el array de arriba, de X[2000]
def car_colour (im):
    x = model.predict(np.array([im]))
    y = enc.inverse_transform([x[0]])[0][0]
    if y=='black':
        return f'The color of the car in the picture is {y}, like my soul'
    elif y== "red":
        return f'The color of the car in the picture is {y}, like the communist you are'
    elif y== "yellow":
        return f'The color of the car in the picture is {y}, like "hielo" in english'
    elif y=="white":
        return f"The color of the car in the picture is {y}, like the white russian I'll have this evening"

In [121]:
print (car_colour (X[7000]))

The color of the car in the picture is white, like the white russian I'll have this evening


In [99]:
X.shape

(10485, 64, 64, 3)

In [ ]:
# Que alguien me explique porque funciona cuando lo hago todo del tiron en la misma funcion 
# pero no cuando lo hago por partes.

#le puedo poner un random choice para que diga distintas estupideces al azar segun el color

# 